## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Biomarker Detection and Classification Pipeline


- **Model**: `en.explain_doc.clinical_biomarker.pipeline`
- **Model Description**: The specialized biomarker pipeline extracts biomarker entities, classifies sentences to identify the presence of biomarker entities, and establishes relations between the extracted entities from clinical documents.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-explain-doc-clinical-biomarker-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [7]:
docs = [
        """The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.""",

        """The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL)."""
]

sample_text = """In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK."""

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data["ner_predictions"])
relation_df = pd.DataFrame(data["relation_predictions"])
classification_df = pd.DataFrame(data["classification_predictions"])

In [9]:
ner_df

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",positive,84,91,Biomarker_Result,0.9672
1,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",CD9,97,99,Biomarker,0.992
2,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",CD10,105,108,Biomarker,0.9987
3,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",tumor markers,151,163,Biomarker,0.48290002
4,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",elevated level,172,185,Biomarker_Result,0.90779996
5,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",Cyfra21-1,190,198,Biomarker,0.9851
6,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",4.77 ng/mL,201,210,Biomarker_Result,0.9719
7,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",NSE,213,215,Biomarker,0.9991
8,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",19.60 ng/mL,218,228,Biomarker_Result,0.96005
9,"In the bone- marrow (BM) asp

In [10]:
relation_df

,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",positive,84,91,Biomarker_Result,CD9,97,99,Biomarker,is_finding_of,0.9932805
1,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",positive,84,91,Biomarker_Result,CD10,105,108,Biomarker,is_finding_of,0.9988914
2,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",tumor markers,151,163,Biomarker,elevated level,172,185,Biomarker_Result,is_finding_of,0.90050846
3,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",tumor markers,151,163,Biomarker,4.77 ng/mL,201,210,Biomarker_Result,O,0.7407979
4,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",tumor markers,151,163,Biomarker,19.60 ng/mL,218,228,Biomarker_Result,O,0.9778502
5,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",tumor markers,151,163,Biomarker,2.58 ng/mL,241,250,Biomarker_Result,O,0.9993332
6,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",elevated level,172,185,Biomarker_Result,Cyfra21-1,190,198,Biomarker,is_finding_of,0.9950375
7,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",elevated level,172,185,Biomarker_Result,NSE,213,215,Biomarker,O,0.81141526
8,"In the bone- m

In [11]:
classification_df

,document,sentence,begin,end,class,class_confidence
0,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.","In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry.",0,127,1,0.99894834
1,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.","Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL.",129,251,1,0.99983305
2,"In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9 and CD10 on flow cytometry. Measurements of serum tumor markers showed elevated level of Cyfra21-1: 4.77 ng/mL, NSE: 19.60 ng/mL, and SCCA: 2.58 ng/mL. Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.","Immunohistochemical staining showed positive staining for CK5/6, P40, and negative staining for TTF-1 and weakly positive staining for ALK.",253,391,1,0.999121


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [12]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data["ner_predictions"])
relation_df = pd.DataFrame(data["relation_predictions"])
classification_df = pd.DataFrame(data["classification_predictions"])

In [13]:
ner_df

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,30,37,Biomarker_Result,0.7702
1,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",EGFR,42,45,Biomarker,None
2,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",exon 21 (L858R,47,60,Biomarker_Result,0.778375
3,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",EGFR-KDD,90,97,Biomarker,0.5694
4,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",cyclin-dependent kinase gene,145,172,Biomarker,0.35023332
5,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,180,187,Biomarker_Result,0.7639
6,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",SMARCA4 gene,196,207,Biomarker,0.7303
7,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",alpha-fetoprotein,177,193,Biomarker,0.9951
8,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",AFP,196,198,Biomarker,0.995
9,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",normal,211,216,Biomarker_Result,0.8999


In [14]:
relation_df

,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,30,37,Biomarker_Result,EGFR,42,45,Biomarker,is_finding_of,0.9997267
1,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,30,37,Biomarker_Result,EGFR-KDD,90,97,Biomarker,is_finding_of,0.76452106
2,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",EGFR,42,45,Biomarker,exon 21 (L858R,47,60,Biomarker_Result,is_finding_of,0.99678254
3,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",exon 21 (L858R,47,60,Biomarker_Result,EGFR-KDD,90,97,Biomarker,O,0.8720176
4,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",cyclin-dependent kinase gene,145,172,Biomarker,mutation,180,187,Biomarker_Result,O,0.7650012
5,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,180,187,Biomarker_Result,SMARCA4 gene,196,207,Biomarker,is_finding_of,0.9763348
6,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",alpha-fetoprotein,177,193,Biomarker,normal,211,216,Biomarker_Result,is_finding_of,0.9780766
7,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",alpha-fetoprotein,177,193,Biomarker,2.2 ng/mL,222,230,Biomarker_Result,is_finding_of,0.8552585
8,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",AFP,196,198,Biomarker,normal,211,216,Biomarker_Result,is_finding_of,0.9946333
9,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",AFP,196,198,Biomarker,2.2 ng/mL,222,230,Biomarker_Result,is_finding_of,0.97450024


In [15]:
classification_df

,document,sentence,begin,end,class,class_confidence
0,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.","The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified.",0,122,0,0.99996203
1,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.,124,229,0,0.99990267
2,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",The ultrasonography revealed the patent's artificial vessels and ascites.,0,72,0,0.9999781
3,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).","The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions.",74,171,0,0.999972
4,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).,173,232,1,0.99928665


### JSON Lines

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [16]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

In [17]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "mutation", "begin": 30, "end": 37, "ner_label": "Biomarker_Result", "ner_confidence": "0.7702"}, {"ner_chunk": "EGFR", "begin": 42, "end": 45, "ner_label": "Biomarker", "ner_confidence": null}, {"ner_chunk": "exon 21 (L858R", "begin": 47, "end": 60, "ner_label": "Biomarker_Result", "ner_confidence": "0.778375"}, {"ner_chunk": "EGFR-KDD", "begin": 90, "end": 97, "ner_label": "Biomarker", "ner_confidence": "0.5694"}, {"ner_chunk": "cyclin-dependent kinase gene", "begin": 145, "end": 172, "ner_label": "Biomarker", "ner_confidence": "0.35023332"}, {"ner_chunk": "mutation", "begin": 180, "end": 187, "ner_label": "Biomarker_Result", "ner_confidence": "0.7639"}, {"ner_chunk": "SMARCA4 gene", "begin": 196, "end": 207, "ner_label": "Biomarker", "ner_confidence": "0.7303"}], "relation_predictions": [{"ner_chunk1": "mutation", "ner_chunk1_begin": "30", "ner_chunk1_end": "37", "ner_label1": "Biomarker_Result", "ner_chunk2": "EGFR", "ner_chunk2_begin": "42", "ner

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [18]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [19]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [20]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [22]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    # Define a dictionary with DataFrame objects and their names
    data_frames = {
        "NER Predictions DataFrame": pd.DataFrame(data['ner_predictions']),
        "Relation Predictions DataFrame": pd.DataFrame(data['relation_predictions']),
        "Classification Predictions DataFrame": pd.DataFrame(data["classification_predictions"])
    }

    # Display and save each DataFrame
    for name, df in data_frames.items():
        print(name + ":")
        display(df)
    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [23]:
process_s3_json_output_and_save(validation_json_file_name)

NER Predictions DataFrame:


,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,30,37,Biomarker_Result,0.7702
1,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",EGFR,42,45,Biomarker,None
2,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",exon 21 (L858R,47,60,Biomarker_Result,0.778375
3,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",EGFR-KDD,90,97,Biomarker,0.5694
4,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",cyclin-dependent kinase gene,145,172,Biomarker,0.35023332
5,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,180,187,Biomarker_Result,0.7639
6,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",SMARCA4 gene,196,207,Biomarker,0.7303
7,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",alpha-fetoprotein,177,193,Biomarker,0.9951
8,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",AFP,196,198,Biomarker,0.995
9,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",normal,211,216,Biomarker_Result,0.8999


Relation Predictions DataFrame:


,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,30,37,Biomarker_Result,EGFR,42,45,Biomarker,is_finding_of,0.9997267
1,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,30,37,Biomarker_Result,EGFR-KDD,90,97,Biomarker,is_finding_of,0.76452106
2,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",EGFR,42,45,Biomarker,exon 21 (L858R,47,60,Biomarker_Result,is_finding_of,0.99678254
3,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",exon 21 (L858R,47,60,Biomarker_Result,EGFR-KDD,90,97,Biomarker,O,0.8720176
4,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",cyclin-dependent kinase gene,145,172,Biomarker,mutation,180,187,Biomarker_Result,O,0.7650012
5,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",mutation,180,187,Biomarker_Result,SMARCA4 gene,196,207,Biomarker,is_finding_of,0.9763348
6,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",alpha-fetoprotein,177,193,Biomarker,normal,211,216,Biomarker_Result,is_finding_of,0.9780766
7,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",alpha-fetoprotein,177,193,Biomarker,2.2 ng/mL,222,230,Biomarker_Result,is_finding_of,0.8552585
8,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",AFP,196,198,Biomarker,normal,211,216,Biomarker_Result,is_finding_of,0.9946333
9,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",AFP,196,198,Biomarker,2.2 ng/mL,222,230,Biomarker_Result,is_finding_of,0.97450024


Classification Predictions DataFrame:


,document,sentence,begin,end,class,class_confidence
0,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.","The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified.",0,122,0,0.99996203
1,"The results revealed that the mutation in EGFR exon 21 (L858R) had disappeared, and a new EGFR-KDD mutation was identified. Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.",Amplification of the cyclin-dependent kinase gene and a mutation in the SMARCA4 gene were also identified.,124,229,0,0.99990267
2,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",The ultrasonography revealed the patent's artificial vessels and ascites.,0,72,0,0.9999781
3,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).","The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions.",74,171,0,0.999972
4,"The ultrasonography revealed the patent's artificial vessels and ascites. The contrast-enhanced computed tomography (CT) scan showed multiple, small, hypervascular lesions. The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).",The alpha-fetoprotein (AFP) level was normal (at 2.2 ng/mL).,173,232,1,0.99928665


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [25]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [26]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "mutation", "begin": 30, "end": 37, "ner_label": "Biomarker_Result", "ner_confidence": "0.7702"}, {"ner_chunk": "EGFR", "begin": 42, "end": 45, "ner_label": "Biomarker", "ner_confidence": null}, {"ner_chunk": "exon 21 (L858R", "begin": 47, "end": 60, "ner_label": "Biomarker_Result", "ner_confidence": "0.778375"}, {"ner_chunk": "EGFR-KDD", "begin": 90, "end": 97, "ner_label": "Biomarker", "ner_confidence": "0.5694"}, {"ner_chunk": "cyclin-dependent kinase gene", "begin": 145, "end": 172, "ner_label": "Biomarker", "ner_confidence": "0.35023332"}, {"ner_chunk": "mutation", "begin": 180, "end": 187, "ner_label": "Biomarker_Result", "ner_confidence": "0.7639"}, {"ner_chunk": "SMARCA4 gene", "begin": 196, "end": 207, "ner_label": "Biomarker", "ner_confidence": "0.7303"}], "relation_predictions": [{"ner_chunk1": "mutation", "ner_chunk1_begin": "30", "ner_chunk1_end": "37", "ner_label1": "Biomarker_Result", "ner_chunk2": "EGFR", "ner_chunk2_begin": "42", "ner

In [27]:
model.delete_model()

INFO:sagemaker:Deleting model with name: explain-clinical-doc-biomarker-2024-05-10-13-48-41-735


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

